# Tutorial on manipulating SLSTR L1 RBT product from the cloud

In [ ]:
import xarray as xr
from pathlib import Path
from xarray_eop.eop import open_eop_datatree


### Get AWS credentials

In [ ]:
import s3fs
import json
# Assuming credentials are in ~/.eopf/secrets.json
try:
    SECRET_PATH = Path.home() / Path(".eopf/secrets.json")
    with open(SECRET_PATH) as f:
        secrets=json.load(f)

# Or use ENV variables
except:
    import os
    secrets = {"s3input" : {
        "key": os.getenv("AWS_ACCESS_KEY_ID"),
        "secret": os.getenv("AWS_SECRET_ACCES_KEY"),
        "endpoint_url": os.getenv("AWS_ENDPOINT_URL"),
        "region_name": os.getenv("AWS_DEFAULT_REGION")
        }
    }

secrets["s3input"].pop("region_name",None)


### Browse S3 buckets and get SLSRBT product

In [ ]:
import s3fs
SAMPLE_PATH = "s3://s3-input/Products/NewFormat/"
s3 = s3fs.S3FileSystem(
    key=secrets["s3input"]["key"],
    secret=secrets["s3input"]["secret"],
    endpoint_url=secrets["s3input"]["endpoint_url"]
)
s3_path = s3.glob(SAMPLE_PATH+"S03SLSRBT*.zip")
store=f"zip::s3://{s3_path[0]}"
store

### Opening the whole product with datatree

In [ ]:
dt = open_eop_datatree(store,backend_kwargs={"storage_options": {"s3":secrets["s3input"]}})
dt

### Opening S7 BT

In [ ]:
s7bt = dt.measurements.inadir.s7_bt_in
s7bt

In [ ]:
import matplotlib.pyplot as plt
bt=s7bt
plt.figure(figsize=(8, 8))
ax = plt.axes()
bt.plot.pcolormesh(
    ax=ax,
    x="longitude", y="latitude", add_colorbar=False
)

### Use the confidence flag to plot land pixels

In [ ]:
group = "quality/inadir"
flag = dt.quality.inadir
meanings=flag.confidence_in.attrs["flag_meanings"].split()
land_bit = meanings.index("land")
land = (flag.confidence_in & 2**land_bit == 2**land_bit)
land.plot()

### Open the meteo data and interpolate the temperature profile at P=980hPa

In [ ]:
meteo=dt.conditions.meteo
meteo.temperature_profile_tp.interp(p_atmos=980).plot()